In [1]:
# timing benchmark
# get info from aapo
# go higher pt, so you know that high pt is good
# project to 1 D and see properties inside (nConstituents, Charged Hadron Fractions)
# ask Andrea for CHS corrections
# timing tools from trigger group
# From scouting meeting
# ~10 ms limit

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.append(os.path.dirname(sys.path[0])) # append project directory to path for import
os.chdir(os.path.dirname(sys.path[0])) # change current working directory to project directory

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import cm

import awkward as ak
import uproot
import hist
import mplhep as hep
import coffea
from coffea import util as cutil
from coffea.lumi_tools import LumiData

import string
import warnings
from functools import partial
import inspect

import itertools
from scipy import optimize
from scipy import stats

from util import *

In [6]:
# configure plot style
hep.style.use("CMS")

data_hep_args={"label":"private", "loc":0, "data":True, "com":13.6}
mc_hep_args={"label":"private", "loc":0, "data":False, "com":13.6}

# global plot parameters
eta_bin_coarse = [0, 1.3, 2.5, 3.0, 5.0]

In [7]:
def get_axis_index(h, name):
    for idx, axis in enumerate(h.axes):
        if axis.name == name:
            return idx
        
def get_string_category_bin_index(h, axis, name):
    for idx, cat in enumerate(h.axes[axis]):
        if cat == name:
            return idx

In [8]:
def compute_stats(h, axis=-1, compute_median=True, compute_mode=False, approx_median=False, approx_mode=False):
    # retrieve axis and bin information
    if isinstance(axis, str):
        axis = get_axis_index(h, axis)
    h_axis = h.axes[axis]
    centers = h_axis.centers
    edges = h_axis.edges
    widths = h_axis.widths
    
    # retrieve counts (np.array)
    freqs = h.counts()
    freqs = np.swapaxes(freqs, axis, -1) # swap to last
    total = np.sum(freqs, axis=-1)
    cmf = np.cumsum(freqs, axis=-1)
    
    # compute mean
    ave = np.sum(centers * freqs, axis=-1) / total
    var = np.sum((centers**2) * freqs, axis=-1) / total - (ave ** 2)
    var = np.where(total > 0, var, 0)
    stdev = np.sqrt(var)
    mean_error = np.where(total > 0, stdev / np.sqrt(total), 0)
    
    # compute median
    median = None
    if compute_median:
        med_bin_idx = np.sum(cmf < np.expand_dims(total/2, axis=-1), axis=-1)
        if approx_median:
            median = centers[med_bin_idx] # approx with center
            median = np.where(total > 0, median, np.nan)
        else:
            med_cmf_before = np.take_along_axis(cmf, np.expand_dims(med_bin_idx-1, axis=-1), axis=-1).squeeze()
            med_freq = np.take_along_axis(freqs, np.expand_dims(med_bin_idx, axis=-1), axis=-1).squeeze()
            med_freq = np.where(med_freq==0, np.nan, med_freq)
            median = edges[med_bin_idx] + (total/2 - med_cmf_before) * widths[med_bin_idx] / med_freq
            
    median_error = 1.2533 * mean_error
    
    # compute mode
    mode = None
    if compute_mode:
        mod_bin_idx = np.argmax(freqs, axis=dep_ax)
        if approx_mode:
            mode = centers[mod_bin_idx] # approx with center
            mode = np.where(total > 0, mode, np.nan)
        if not approx_mode: # experimental
            mod_freq = np.take_along_axis(freqs, np.expand_dims(mod_bin_idx, axis=-1), axis=-1).squeeze()
            diff_low = mod_freq - np.take_along_axis(freqs, np.expand_dims(mod_bin_idx-1, axis=-1), axis=-1).squeeze()
            diff_high = mod_freq - np.take_along_axis(freqs, np.expand_dims(mod_bin_idx+1, axis=-1), axis=-1).squeeze()
            mode = edges[mod_bin_idx] + (diff_low / (diff_low + diff_high)) * widths[med_bin_idx]
    
    freqs = np.swapaxes(freqs, axis, -1) # swap back
            
    return {"centers": centers, "freqs": freqs, "mean": ave, \
            "stdev": stdev, "var": var, "mean_error": mean_error, \
            "median": median, "median_error": median_error, \
            "mode": mode}

In [9]:
# histogram manipulation

def make_single_string_category_axis(cat, name, label=None):
    assert isinstance(category, str)
    label = label if label is not None else name
    return hist.axis.StrCategory([category], name=name, label=label)

def add_histogram_axis(h, axis_index, new_axis):
    assert isinstance(new_axis, hist.axis.StrCategory), "New axis must be string category axis"
    assert len(new_axis) == 1, "New axis must have exactly one category"
    new_axes = list(h.axes)
    new_axes.insert(axis_index, new_axis)
    new_hist = hist.Hist(*new_axes, storage=h.storage_type())
    new_hist[{new_axis.name: new_axis[0]}] = h.view(flow=True)
    return new_hist

def merge_histograms_existing_axis(*hs, axis="dataset"):
    if len(hs) == 1: # if only one histogram is given, just return
        return hs[0]
    # checking
    assert all([isinstance(h.axes[axis], hist.axis.StrCategory) for h in hs]), \
            "Merge axis must be string category axis"
    assert all([len(h.axes) == len(hs[0].axes) for h in hs]), "Other axes must match"
    assert all([ax == hs[i].axes[ax.name] for h in hs for ax in h.axes if ax.name != axis for i in range(len(hs))]), \
           "Other axes must match"
    
    # build new axes
    merged_categories = [cat for h in hs for cat in h.axes[axis]]
    merged_axis = hist.axis.StrCategory(merged_categories, name=axis, label=hs[0].axes[axis].label, growth=True)
    merged_axes = list(hs[0].axes)
    merged_axes[get_axis_index(hs[0], axis)] = merged_axis # insert new axis
    
    # create merged histogram
    merged_hist = hist.Hist(*merged_axes, storage=hs[0].storage_type())
    
    # fill merged histogram
    for h in hs:
        for cat in h.axes[axis]: 
            merged_hist[{axis: cat}] = h[{axis:cat}].view(flow=True)
        
    return merged_hist

def merge_histograms_new_axis(h_dict, name="untitled", label=None, index=0):
    if len(h_dict) == 1: # if only one histogram is given, just return
        return list(h_dict.values())[0]
    
    label = label if label is not None else name
    h_dict_items = h_dict.items()
    cats, hs = list(zip(*h_dict_items))
    assert all([len(h.axes) == len(hs[0].axes) for h in hs]), "All axes must match"
    assert all([ax == hs[i].axes[ax.name] for h in hs for ax in h.axes for i in range(len(hs))]), \
           "All axes must match"
    # build new axis
    new_axis = hist.axis.StrCategory(cats, name=name, label=label, growth=True)
    merged_axes = list(hs[0].axes)
    merged_axes.insert(index, new_axis)
    
    # create merged histogram
    merged_hist = hist.Hist(*merged_axes, storage=hs[0].storage_type())
    
    # fill merged histogram
    for h in hs:
        for cat, h in h_dict_items: 
            merged_hist[{name: cat}] = h.view(flow=True)
    return merged_hist

def merge_histograms_from_stack(h_stack, name="untitled", label=None, index=0):
    h_dict = {h.name:h for h in h_stack}
    return merge_histograms_new_axis(h_dict, name=name, label=label, index=index)
    
def merge_histograms(*hs, axis="dataset", label=None, index=0):
    if len(hs) == 1 and isinstance(hs[0], dict):
        return merge_histograms_new_axis(hs[0], name=axis, label=label, index=index)
    elif len(hs) == 1 and isinstance(hs[0], hist.stack.Stack):
        return merge_histograms_from_stack(hs[0], name=axis, label=label, index=index)
    else:
        if axis in hs[0].axes.name:
            return merge_histograms_existing_axis(*hs, axis=axis)
        else:
            # create h_dict with default dict keys
            h_dict = {str(i):h for i, h in enumerate(hs)}
            return merge_histograms_new_axis(h_dict, name=axis, label=label, index=index)
        
def reduce_histogram(h, reduced_axes):
    assert all([axis in h.axes.name for axis in reduced_axes]) # reduced_axes must be the subset of original axes
    reduction_dict = {axis:sum for axis in h.axes.name if axis not in reduced_axes}
    return h[reduction_dict]

def integrate_histogram(h, axis, upper=None, lower=0, mirror=False): # experimental, please use properly
    if upper is None:
        if lower is None:
            return h[{axis: sum}]
        else:
            if mirror:
                return h - h[{axis: slice(-lower * 1j, lower*1j,sum)}]
            else:
                pass
                #return h - h[{axis: slice(-np.inf, lower*1j,sum)}]
    
    upper, lower = (upper, lower) if upper > lower else (lower, upper)
    rslt = h[{axis: slice(lower * 1j, upper*1j,sum)}]
    if not mirror:
        return rslt
    else:
        return rslt + h[{axis: slice(-upper * 1j, -lower*1j,sum)}]

In [10]:
# merge three response histograms together
def merge_response_histograms(out, dataset):
    response_name=["raw_pt_response", "original_pt_response", "corrected_pt_response"]
    h_resp_list = [out[resp][{"dataset":dataset}] for resp in response_name]
    h_resp_dict = {h.label:h for h in h_resp_list}
    h_resp_stack = hist.Stack.from_dict(h_resp_dict)
    return merge_histograms(h_resp_stack, axis="response", label="Response")

In [11]:
# common selection and integrate
def preprocess_histogram(h, dataset=None, correction_level=None, jet_type=None, 
                          eta_range=None, phi_range=None, pt_range=None):
    # build selection dict
    selection_dict = dict()
    if dataset:
        selection_dict["dataset"] = dataset
    if correction_level:
        selection_dict["correction_level"] = correction_level
    if jet_type:
        selection_dict["jet_type"] = jet_type
    if eta_range == sum:
        selection_dict["jet_eta"] = eta_range
    if phi_range == sum:
        selection_dict["jet_phi"] = eta_range
    if pt_range == sum:
        selection_dict["jet_pt"] = eta_range
    
    # apply selection dict
    h = h[selection_dict]
    
    # integrate
    if eta_range and eta_range != sum:
        if len(eta_range) == 2:
            eta_range = eta_range + (True,) # mirror integrate
        h = integrate_histogram(h, "jet_eta", *eta_range)
    if phi_range and phi_range != sum:
        if len(phi_range) == 2: 
            phi_range = phi_range + (False,)
        h = integrate_histogram(h, "jet_phi", *phi_range)
    if pt_range and pt_range != sum:
        if len(pt_range) == 2:
            pt_range = pt_range + (False,)
        h = integrate_histogram(h, "jet_pt", *pt_range)
        
    return h

In [12]:
# plot utilities

def strike(text):
    result = ""
    for c in text:
        result = result + c + "\u0336"
    return result
    
def format_range_text(lower, upper, var, absolute=False, omit_zero=True):
    if lower > upper:
        lower, upper = upper, lower
    if absolute:
        var = "|{}|".format(var)
    if omit_zero and lower == 0:
        return "{} < {}".format(var, upper)
    else:
        return "{} < {} < {}".format(lower, var, upper)

def format_eta_range_text(lower, upper, jet_name="jet", absolute=True, omit_zero=False):
    if jet_name:
        return format_range_text(lower, upper, "$\eta^{%s}$"%(jet_name), absolute=absolute, omit_zero=omit_zero)
    else:
        return format_range_text(lower, upper, "$\eta$", absolute=absolute, omit_zero=omit_zero)

def format_pt_range_text(lower, upper, jet_name="jet", omit_zero=True):
    return format_range_text(lower, upper, "$p_T^{%s}$"%(jet_name), omit_zero=omit_zero)

def format_phi_range_text(lower, upper, jet_name="jet", omit_zero=False):
    return format_range_text(lower, upper, "$\phi^{%s}$"%(jet_name), omit_zero=omit_zero)

# save figure to file
def save_figure(fig, filename=None, dpi=100):
    if filename:
        fig.savefig(filename, bbox_inches="tight", dpi=dpi)

# wrapper to call appropriate matplotlib functions
def format_plt_plot(ax,
                    xscale=None, yscale=None,
                    xlim=None, ylim=None,
                    legend_title=None, legend_loc=0, legend_args=None):
    
    # axis scaling
    if xscale:
        ax.set_xscale(xscale)
    if yscale:
        ax.set_yscale(yscale)
    
    # setting axis limit
    # left=None, right=None, *, emit=True, auto=False, xmin=None, xmax=None
    if xlim:
        if isinstance(xlim, set) or isinstance(xlim, tuple):
            ax.set_xlim(*xlim)
        if isinstance(xlim, dict):
            ax.set_xlim(**xlim)
    if ylim:
        if isinstance(ylim, set) or isinstance(ylim, tuple):
            ax.set_ylim(*ylim)
        if isinstance(ylim, dict):
            ax.set_ylim(**xlim)
    
    # formatting legend
    if legend_title:
        ax.legend(title=legend_title, loc=legend_loc)
    if legend_args: # other arguments, see matplotlib document for reference
        if legend_title and "title" not in legend_args:
            legend_args["title"] = legend_title
        if legend_loc and "loc" not in legend_args:
            legend_args["loc"] = legend_loc
        ax.legend(**legend_args)

# wrapper to call appropriate mplhep functions
def format_hep_plot(ax,
                    hep_args=None, with_cms_name=True,
                    hep_magic=False):
    if hep_args:
        if with_cms_name:
            hep.cms.label(**hep_args)
        else: # experimental, x, y, and fontsize may need to be adjusted manually
            if hep_args["data"]:
                ax.set_title("private", style="italic", ha="left", x=0, y=1.005, fontsize=28)
            else:
                ax.set_title("Simulation private", style="italic", ha="left", x=0, y=1.005, fontsize=28)
                
    if hep_magic:
        try:
            hep.mpl_magic(ax=ax)
        except:
            warnings.warn("mplhep magic fails! trying yscale_legend")
            try:
                hep.plot.yscale_legend(ax=ax)
            except:
                warnings.warn("mplhep yscale also fails!")

# combine both wrappers and save figure                
def format_plot(fig, ax,
                xscale=None, yscale=None,
                xlim=None, ylim=None,
                legend_title=None, legend_loc=0, legend_args=None, 
                hep_args=None, with_cms_name=True,
                hep_magic=False,
                filename=None, dpi=100):
    
    format_plt_plot(ax, xscale=xscale, yscale=yscale, xlim=xlim, ylim=ylim, 
                    legend_title=legend_title, legend_loc=legend_loc, legend_args=legend_args)
    
    if hep_args or hep_magic:
        format_hep_plot(ax, hep_args=hep_args, with_cms_name=with_cms_name, hep_magic=hep_magic)
        
    save_figure(fig, filename, dpi)

## Load Coffea output

In [14]:
out_dir = "/ssd-home/pinkaew/research/online-offline-jec/coffea_output"
out_filename = "runD_AK4Puppi_HLT_notrigger_notagprobe.coffea"
#out_filename="runD_AK4Puppi_HLT_notrigger_notagprobe_noMETfilters_comparison.coffea"
#out_filename="runD_AK4Puppi_HLT_notrigger_tagprobe20_comparison.coffea"
#out_filename="mc_AK4Puppi_HLT_notrigger_notagprobe_all.coffea"
#out_filename="mc_AK4Puppi_HLT_notrigger_notagprobe_noMETfilters_all.coffea"
#out_filename="mc_AK4Puppi_HLT_notrigger_tagprobe20_all.coffea"
#out_filename="mc_Gen_HLT_notrigger_notagprobe_all.coffea"
#out_filename="mc_HLT_Gen_notrigger_notagprobe_all.coffea"
#out_filename="mc_Gen_AK4Puppi_notrigger_notagprobe_all.coffea"
#out_filename = "runF_AK4Puppi_HLT_notrigger_notagprobe_comparison.coffea"
#out_filename = "runF_AK4Puppi_HLT_notrigger_notagprobe_noMETFilters.coffea" # nothing
#out_filename = "runD_AK4Puppi_HLT_notrigger_notagprobe_lxplus_v2.coffea"
#out_filename = "runE_AK4Puppi_HLT_notrigger_notagprobe_lxplus.coffea"
#out_filename = "runF_AK4Puppi_HLT_notrigger_notagprobe_lxplus_rerun.coffea"
#out_filename = "mc_AK4Puppi_HLT_notrigger_notagprobe_unweighted.coffea"
#out_filename = "mc_AK4Puppi_HLT_notrigger_notagprobe_unweighted_response.coffea"
#out_filename = "mc_AK4Puppi_HLT_notrigger_notagprobe_weighted.coffea"
#out_filename = "mc_AK4Puppi_HLT_notrigger_notagprobe_weighted_response.coffea"
#out_filename = "runF_AK4Puppi_Scouting_notrigger_notagprobe.coffea"
#out_filename = "ScoutingPFMonitor_2022F_AK4Puppi_Scouting_notrigger_notagprobe.coffea"
#out_filename = "ScoutingPFMonitor_2022F_AK4Puppi_Scouting_pvmatched_notrigger_notagprobe.coffea"
#out_filename = "ScoutingPFMonitor_2022F_AK4Puppi_HLT_notrigger_notagprobe.coffea"
#out_filename = "JetMET_2022F_AK4Puppi_HLT_notrigger_notagprobe.coffea"
#out_filename = "ScoutingPFMonitor_2022F_AK4Puppi_Scouting_notrigger_notagprobe_sameetabin.coffea"
out = coffea.util.load(os.path.join(out_dir, out_filename))

In [15]:
print(out.keys())
print_dict_json(out["arguments"], "arguments")
print_dict_json(out["configurations"], "configurations")

dict_keys(['cutflow', 'processed_lumi', 'pt_response', 'pt_percent_difference', 'comparison', 'jet_pt', 'jet_eta', 'jet_phi', 'time_pf', 'arguments', 'configurations', 'start_timestamp', 'process_time'])
arguments
{
    "input_dir": [
        "/data/data/JME_NANO_DATA/2022/JMENanoRun3_v2p1_Run2022D-PromptReco-v2/JetMET"
    ],
    "dataset_name": [
        "JetMET"
    ],
    "out_file": "coffea_output/runD_AK4Puppi_HLT_notrigger_notagprobe.coffea",
    "config_file": "configs/runD_AK4Puppi_HLT_notrigger_notagprobe.cfg",
    "trigger_type": null,
    "trigger_min_pt": null,
    "off_jet_tag_probe": null,
    "on_jet_tag_probe": null,
    "off_jet_tag_min_pt": null,
    "on_jet_tag_min_pt": null
}
configurations
{
    "Processor": {
        "off_jet_name": "Jet",
        "off_jet_label": "AK4PUPPI",
        "on_jet_name": "TrigObjJMEAK4",
        "on_jet_label": "AK4HLT",
        "lumi_json_path": "corrections/lumimask/Cert_Collisions2022_eraD_357538_357900_Golden.json",
        "lumi_c

In [16]:
out["process_time"]

datetime.timedelta(seconds=6928, microseconds=489662)

In [20]:
try:
    dataset = out["configurations"]["IO"]["dataset_names"][0]
except:
    dataset = out["arguments"]["dataset_name"][0]
dataset

'JetMET'

## Luminosity

In [22]:
from coffea.lumi_tools import LumiData
data_taking_era = "2022D"

lumidata = LumiData("corrections/lumidata/lumi{}.csv".format(data_taking_era))
out["processed_lumi"][dataset]["lumi_list"].unique() # apply unique
out["processed_lumi"][dataset]["lumi"] = lumidata.get_lumi(out["processed_lumi"][dataset]["lumi_list"])

In [23]:
# pick hep_args
is_data = eval(out["configurations"]["Processor"]["is_data"])
if is_data:
    print("luminosity: {:.3f}".format(out["processed_lumi"][dataset]["lumi"]))
    data_hep_args["lumi"] = "{:.3f}".format(out["processed_lumi"][dataset]["lumi"])
    
data_hep_args["year"] = data_taking_era
hep_args = data_hep_args if is_data else mc_hep_args

luminosity: 1.781


In [24]:
hep_args

{'label': 'private',
 'loc': 0,
 'data': True,
 'com': 13.6,
 'lumi': '1.781',
 'year': '2022D'}

## Cutflow

In [ ]:
def plot_cutflow(cutflow, hep_args=None, legend_title=None, filename=None):
    fig, ax = plt.subplots()
    x = np.arange(len(cutflow))
    y = cutflow.values()
    xlabels = cutflow.keys()
    ax.step(x, y, "|-", where="post")
    ax.set_xticks(x, xlabels, rotation = "vertical")
    for i, v in enumerate(y):
        ax.text(x[i]+0.05, v + 100, str(v), fontsize=10)
        ax.axvline(x[i], linestyle="dashed", alpha=0.1)
    
    format_plot(fig, ax, legend_title=legend_title, legend_loc="upper right", 
                hep_args=hep_args, hep_magic=False,
                filename=filename)

In [ ]:
out["cutflow"]#[dataset] #4578615

In [ ]:
#%%script false --no-raise-error
plot_cutflow(out["cutflow"][dataset], hep_args=hep_args)

## Offline vs Online 2D plot

In [29]:
#from sklearn.decomposition import PCA

# re-implement methods from TH2
def get_stats(h):
    assert len(h.axes.name) == 2, "Histogram must be two dimensional"
    w = h.counts()
    x = np.expand_dims(h.axes[0].centers, axis=1)
    y = np.expand_dims(h.axes[1].centers, axis=0)
    sumw = np.sum(w)
    sumw2 = None
    sumwx = np.sum(w*x)
    sumwx2 = np.sum(w*(x*x))
    sumwy = np.sum(w*y)
    sumwy2 = np.sum(w*(y*y))
    sumwxy = np.sum((w*x)*y)
    
    return {"sumw": sumw, "sumw2": None, 
            "sumwx": sumwx, "sumwx2":sumwx2, 
            "sumwy": sumwy, "sumwy2": sumwy2, 
            "sumwxy": sumwxy}

def get_covariance(h):
    assert len(h.axes.name) == 2, "Histogram must be two dimensional"
    stats = get_stats(h)
    return stats["sumwxy"]/stats["sumw"] - stats["sumwx"]/stats["sumw"]*stats["sumwy"]/stats["sumw"]

def get_correlation_factor(h):
    assert len(h.axes.name) == 2, "Histogram must be two dimensional"
    stats = get_stats(h)
    stdevx = np.sqrt(stats["sumwx2"]/stats["sumw"] - (stats["sumwx"]/stats["sumw"])**2)
    stdevy = np.sqrt(stats["sumwy2"]/stats["sumw"] - (stats["sumwy"]/stats["sumw"])**2)
    cov = stats["sumwxy"]/stats["sumw"] - stats["sumwx"]/stats["sumw"]*stats["sumwy"]/stats["sumw"]
    return cov/stdevx/stdevy

def repeat_value_from_freq(values, freqs):
    return list(itertools.chain.from_iterable([itertools.repeat(elem, n) for elem, n in zip(values, freqs)]))

def build_points_from_histogram(h):
    assert len(h.axes.name) == 2, "Histogram must be two dimensional"
    w = h.counts()
    x = h.axes[0].centers
    y = h.axes[1].centers
    values = itertools.product(x, y)
    points = np.array(repeat_value_from_freq(values, w.reshape(-1)))
    return points

def compute_correlation(h):
    points = build_points_from_histogram(h)
    return stats.pearsonr(points[:, 0], points[:, 1])[0]

def compute_pca(h):
    points = build_points_from_histogram(h)
    return PCA(n_components=2).fit(points)

# example for converting to root
# root_file = uproot.recreate("test_uproot_file.root")
# root_file["hist"] = h_test.to_numpy()

# histFile = ROOT.TFile.Open("test_uproot_file.root")
# dataHisto = histFile.Get("hist")

# dataHisto.GetCorrelationFactor()

In [30]:
def format_correction_level(off_correction_level, on_correction_level=None):
    if not on_correction_level:
        on_correction_level = off_correction_level
        if len(off_correction_level.split(":")) == 2: # already in correct format
            return off_correction_level
    return "off={}:on={}".format(off_correction_level, on_correction_level)

def extract_correction_level(correction_level):
    off_correction_level, on_correction_level = correction_level.split(":")
    return off_correction_level.split("=")[1], on_correction_level.split("=")[1]

In [31]:
def plot_online_offline_pt(out, dataset="QCD", correction_level="Raw", 
                           eta_range=None, phi_range=None,
                           trigger_value=None, trigger_values=None,
                           xscale=None, yscale=None, xlim=None, ylim=None,
                           legend_title=None, 
                           hep_args=None, 
                           filename=None, dpi=100):
    fig, ax = plt.subplots()
    cmap = cm.get_cmap('rainbow')
    #cmap.set_bad("k")
    cmap.set_under("w")
    
    off_jet_label = out["configurations"]["Processor"]["off_jet_label"]
    on_jet_label = out["configurations"]["Processor"]["on_jet_label"]
    correction_level = format_correction_level(correction_level) # TODO: need to fix this
    
    
    h = preprocess_histogram(out["comparison"], dataset=dataset,
                             correction_level=correction_level,
                             jet_type=off_jet_label, # + " (Matched Gen)", 
                             eta_range=eta_range, phi_range=phi_range)
    h = h.project("off_jet_pt", "on_jet_pt")
#     h_np = h.values() # convert to np
#     h_np_norm = np.where(h_np < 1e7, h_np, 0)
#     h_norm = hist.Hist(*h.axes, storage=hist.storage.Double())
#     h.view()[:] = h_np_norm # convert back to hist
    h.plot2d(ax=ax, cmap=cmap, alpha=1, norm=colors.LogNorm())
    
    
    # compute correlation
    corr = get_correlation_factor(h)
    
    # compute pca and plot first principal component (PC1)
#     pca = compute_pca(h)
#     pca_comp0 = pca.components_[0]
#     pca_var0 = pca.explained_variance_[0]
#     pca_scaled_comp0 = pca_comp0 * pca_var0  # scale component by its variance explanation power
#     ax.plot([0, pca_scaled_comp0[0]], [0, pca_scaled_comp0[1]], 
#              label="PC1", linewidth=1.5, color="black")
    
    # plot y = x
    ax.axline([1, 1], slope=1, color="k", linestyle="dashed")
    ax.text(1e4, 8e3, "y=x", rotation=45, fontsize=20)
    
    # plot x=trigger_values and y=trigger_values
    if trigger_values:
        for value in trigger_values:
            ax.axhline(y=value, color="k", linestyle="dotted", alpha=0.3)
            ax.axvline(x=value, color="k", linestyle="dotted", alpha=0.3)
    
    # plot x=trigger_values and y=trigger_values (highlight)
    if trigger_value:
        ax.axhline(y=trigger_value, color="k", linestyle="dotted", alpha=1)
        ax.axvline(x=trigger_value, color="k", linestyle="dotted", alpha=1)
        
    legend_title_list = [dataset]
    if eta_range:
        legend_title_list.append(format_eta_range_text(eta_range[0], eta_range[1], 
                                                       jet_name=None, omit_zero=True))
    legend_title_list.append("corr: {:.5f}".format(corr))
    if legend_title:
        legend_title = "\n".join([legend_title] + legend_title_list)
    else:
        legend_title = "\n".join(legend_title_list)
        
    # TODO: need to move to better place...
    off_correction_level, on_correction_level = extract_correction_level(correction_level)
    off_jet_pt_axis_label = "\ ".join([off_jet_label, "p_T", "({})".format(off_correction_level)])
    on_jet_pt_axis_label = "\ ".join([on_jet_label, "p_T", "({})".format(on_correction_level)])
    ax.set_xlabel(r"${}$".format(off_jet_pt_axis_label))
    ax.set_ylabel(r"${}$".format(on_jet_pt_axis_label))
    
    # format and save plot
    format_plot(fig, ax, xscale=xscale, yscale=yscale, xlim=xlim, ylim=ylim, 
                legend_title=legend_title, legend_args={"title_fontsize":20, "fontsize": 20},
                hep_args=hep_args,
                filename=filename, dpi=dpi)

In [ ]:
plot_online_offline_pt(out, dataset=dataset, correction_level= "Raw", #"Original", #"off=Raw:on=Raw",
                       legend_title= "Notitled", #"pvmatched\nw/o T&P, w/ METfilters",
                       xscale="log", yscale="log", xlim=(0, 6e4), ylim=(0, 6e4), 
                       hep_args=hep_args)

In [ ]:
def plot_multiple_eta_bin(func, eta_bin_plot, func_args, mirror=True):
    for eta_plot_idx in range(len(eta_bin_plot)-1):
        eta_low = eta_bin_plot[eta_plot_idx]
        eta_high = eta_bin_plot[eta_plot_idx+1]
        func(**func_args, eta_range=(eta_low, eta_high, mirror))

In [ ]:
func_args = {"out": out, 
             "dataset": dataset, "correction_level": "Raw", #"Original", #"off=Raw:on=Raw",
             "legend_title":"w/o T&P, w/ METFilters",
             "xscale":"log", "yscale":"log", "xlim":(0, 6e4), "ylim":(0, 6e4), 
             "hep_args":hep_args}

In [ ]:
plot_multiple_eta_bin(plot_online_offline_pt, eta_bin_plot=eta_bin_coarse, func_args=func_args, mirror=True)

## Plot pt response vs pt

In [ ]:
def plot_response_pt(out, dataset="QCD", correction_level="Original", 
                     jet_type=None,
                     eta_range=None, phi_range=None, # if range None, will integrate all
                     normalize_pt=False, plot_what={"2d", "profile"},
                     fig = None, ax = None,
                     xscale=None, yscale=None, xlim=None, ylim=None,
                     legend_title=None, hep_args=None, filename=None):
    if fig == None or ax == None:
        fig, ax = plt.subplots()
    size = fig.get_size_inches()
    fig.set_size_inches((size[0]*1.5, size[1]))
    
    cmap = cm.get_cmap('rainbow')
    cmap.set_under('w')
    
    on_jet_label = out["configurations"]["Processor"]["on_jet_label"]
    off_jet_label = out["configurations"]["Processor"]["off_jet_label"]
    correction_level = format_correction_level(correction_level)
    if not jet_type:
        jet_type = off_jet_label
        
    h = preprocess_histogram(out["pt_response"], dataset=dataset, correction_level=correction_level, \
                            jet_type=jet_type, eta_range=eta_range, phi_range=phi_range)
    
    h = (h.project("jet_pt", "pt_response"))[0j:, :3.5j]
    
    if plot_what == None:
        plot_what = {"2d", "profile"}
    
    if "2d" in plot_what:
        if normalize_pt:
            h_np = h.values() # convert to np
            h_np_norm = h_np / np.sum(h_np, axis=1, keepdims=True)
            h_norm = hist.Hist(*h.axes, storage=hist.storage.Double())
            h_norm.view()[:] = h_np_norm # convert back to hist
            h_norm.plot2d(ax=ax, cmap=cmap, alpha=1, norm=colors.LogNorm())

        else:
            h.plot2d(ax=ax, cmap=cmap, alpha=1, norm=colors.LogNorm())
    
    # compute profile and plot profile
    if "profile" in plot_what:
        hp = h.profile("pt_response")
        hp.plot(color="k", linewidth=1.5)
        ax.plot(hp.axes[0].centers, hp.to_numpy()[0], color="k", linestyle="", marker="^", markersize=3)
    
    # plot pt_response = 1.0 line
    ax.axhline(1, linestyle="dashed", color="k", alpha=0.5)
    
    ax.set_xlabel(r"$p_T^{%s}$"%off_jet_label)
    ax.set_ylabel(r"$p_T^{%s} / p_T^{%s}$"%(on_jet_label, off_jet_label))
    
    # TODO: need to move to better place...
    off_correction_level, on_correction_level = extract_correction_level(correction_level)
    off_jet_pt_axis_label = "\ ".join([off_jet_label, "p_T", "({})".format(off_correction_level)])
    on_jet_pt_axis_label = "\ ".join([on_jet_label, "p_T", "({})".format(on_correction_level)])
    if jet_type.startswith(off_jet_label):
        jet_type_correction_level = off_correction_level
    else:
        jet_type_correction_level = on_correction_level
    jet_type_pt_axis_label = "\ ".join([*jet_type.split("_"), "p_T", "({})".format(jet_type_correction_level)])
    ax.set_xlabel(r"${}$".format(jet_type_pt_axis_label))
    ax.set_ylabel(r"$R = \frac{%s}{%s}$"%(str(on_jet_pt_axis_label), str(off_jet_pt_axis_label)))
    
    legend_title_list = [dataset]
    if eta_range:
        legend_title_list.append(format_eta_range_text(eta_range[0], eta_range[1], 
                                                       jet_name="\ ".join(jet_type.split("_")), omit_zero=True))
    if legend_title:
        legend_title = "\n".join([legend_title] + legend_title_list)
    else:
        legend_title = "\n".join(legend_title_list)
    
    format_plot(fig, ax, xscale=xscale, yscale=yscale, xlim=xlim, ylim=ylim,
                legend_title=legend_title, hep_args=hep_args, filename=filename)
    
    return fig, ax

In [ ]:
out["pt_response"]

In [ ]:
plot_response_pt(out, dataset=dataset, 
                 #correction_level="off=Original:on=Raw",
                 #correction_level="off=Raw:on=Original",
                 correction_level="Original",
                 jet_type="Gen",
                 eta_range=(1.3, 2.5, True), 
                 phi_range=None,
                 xlim=(0, None),
                 xscale="log", normalize_pt=False,
                 legend_title="No Tag and probe",
                 hep_args=hep_args,
                 #filename="results/response_pt_corrected_notrigger_notagprobe_mc_density_xlog.png"
                )

In [ ]:
func_args = {"out":out, 
             "dataset":dataset, 
             #correction_level="off=Original:on=Raw",
             #correction_level="off=Raw:on=Original",
             "correction_level":"Original",
             "jet_type":"Ave (Matched Gen)",
             "phi_range":None,
             "xlim":(0, None),
             "xscale":"log", 
             "normalize_pt":False,
             "legend_title":"Weighted\nNo Tag and probe",
             "hep_args":hep_args,
             #filename="results/response_pt_corrected_notrigger_notagprobe_mc_density_xlog.png"
            }
plot_multiple_eta_bin(plot_response_pt, eta_bin_plot=eta_bin_coarse, func_args=func_args, mirror=True)

## Fit pt response in each pt-eta bin

In [32]:
def norm(x, A, mu, sigma):
    return A * stats.norm.pdf(x, mu, sigma)

def gamma(x, A, a, loc, scale):
    return A * stats.gamma.pdf(x, a, loc, scale)

def lognormal(x, A, mu, sigma, scale):
    return A * stats.lognorm.pdf(x, sigma, mu, scale)

def gumbel_r(x, A, loc, scale):
    return A * stats.gumbel_r.pdf(x, loc, scale)

def nct(x, A, df, nc, loc, scale):
    return A * stats.nct.pdf(x, df, nc, loc, scale)

In [60]:
def fit_response(out, dataset="QCD", correction_level="Original", jet_type="PUPPI",
                 phi_range=None, # if range None, will integrate all
                 plot=False,
                 xscale="linear", yscale="linear", xlim=None, ylim=None,
                 legend_title=None, hep_args=None, filename=None, dpi=100):
    
    correction_level = format_correction_level(correction_level)
    h = preprocess_histogram(out["pt_response"], dataset=dataset, correction_level=correction_level, \
                        jet_type=jet_type, phi_range=phi_range)
    h = h.project("jet_pt", "jet_eta", "pt_response")
    
    h_stats = compute_stats(h)
    h_stats["fit_mean"] = np.nan * np.ones_like(h_stats["mean"])
    h_stats["fit_stdev"] = np.nan * np.ones_like(h_stats["stdev"])
    h_stats["fit_var"] = np.nan * np.ones_like(h_stats["var"])
    h_stats["fit_mean_error"] = np.nan * np.ones_like(h_stats["mean_error"])
    
    return h_stats
    
    if plot:
        x_plot = np.linspace(h.axes[-1].edges[0], h.axes[-1].edges[-1], 500)
    
    for pt_bin_idx in range(len(h.axes[0])):
        pt_range_text = format_pt_range_text(*h.axes[0][pt_bin_idx], jet_name=jet_type, omit_zero=False)
        for eta_bin_idx in range(len(h.axes[1])):
            eta_range_text = format_eta_range_text(*h.axes[1][eta_bin_idx], jet_name=jet_type, \
                                                   absolute=False, omit_zero=False)
            
            h_resp = h[pt_bin_idx, eta_bin_idx, :]
            xdata = h_stats["centers"]
            ydata = h_stats["freqs"][pt_bin_idx, eta_bin_idx]
            yerr = np.sqrt(h_resp.variances())
            mask = (yerr != 0.0)
            xdata = xdata[mask]
            ydata = ydata[mask]
            yerr = yerr[mask]
            total = np.sum(ydata)
            num_nonzero_bins = np.sum(ydata > 0)
            
            if (num_nonzero_bins < 2) or (total < 100):
                continue
            print(num_nonzero_bins, total)  
            # need to update this
            # inspired from hist.plot._fit_callable_to_hist
            p0 = [total, h_stats["mean"][pt_bin_idx, eta_bin_idx], h_stats["stdev"][pt_bin_idx, eta_bin_idx]]
            popt, pcov = optimize.curve_fit(norm, xdata, ydata, sigma=yerr, absolute_sigma=True, p0=p0)
            
            _, fit_mean, fit_stdev = popt
            fit_mean_error = np.sqrt(pcov[1, 1])
            
            # populate stats dict
            h_stats["fit_mean"][pt_bin_idx, eta_bin_idx] = fit_mean
            h_stats["fit_stdev"][pt_bin_idx, eta_bin_idx] = fit_stdev
            h_stats["fit_var"][pt_bin_idx, eta_bin_idx] = fit_stdev ** 2
            h_stats["fit_mean_error"][pt_bin_idx, eta_bin_idx] = fit_mean_error
            
            if plot:
                if not (pt_bin_idx == 28 and eta_bin_idx == 69):
                    continue
                
                fig, ax = plt.subplots()
                h_resp.plot(ax=ax) # plot histogram

                y_plot = norm(x_plot, *popt) # plot gaussian fit

                # for log y scale, plot only y > 1
                if yscale == "log":
                    ax.plot(x_plot[y_plot>=1], y_plot[y_plot>1])
                else:
                    ax.plot(x_plot, y_plot)
                    
                # add y axis label
                ax.set_ylabel("Events")

                # format legend
                legend_title = "" if not legend_title else legend_title + "\n"
                legend_title += "{}\n{}\n\n".format(pt_range_text, eta_range_text)
                legend_title += "count: {}\n".format(total)
                legend_title += "mean: {:.3f}\nstdev: {:.3f}\nmean_err: {:.3f}\n"\
                                .format(h_stats["mean"][pt_bin_idx, eta_bin_idx], 
                                        h_stats["stdev"][pt_bin_idx, eta_bin_idx], 
                                        h_stats["mean_error"][pt_bin_idx, eta_bin_idx])
                legend_title += "median: {:.3f}\nmedian_err: {:.3f}\n"\
                                .format(h_stats["median"][pt_bin_idx, eta_bin_idx],
                                        h_stats["median_error"][pt_bin_idx, eta_bin_idx])
                legend_title += "fit_mean: {:.3f}\nfit_stdev: {:.3f}\nfit_mean_err: {:.3f}"\
                                .format(fit_mean, fit_stdev, fit_mean_error)

                # TODO: write filename formatting for saving, use only None for now
                format_plot(fig, ax, xscale=xscale, yscale=yscale, xlim=xlim, ylim=ylim, 
                            legend_title=legend_title, legend_args={"title_fontsize":20},
                            hep_args=hep_args,
                            filename=filename, dpi=dpi)
            
    return h_stats

In [61]:
out["pt_response"]

Hist(
  StrCategory(['JetMET'], growth=True, name='dataset', label='Primary Dataset'),
  StrCategory(['off=Original:on=Original'], name='correction_level', label='Correction levels'),
  StrCategory(['AK4PUPPI', 'AK4HLT'], name='jet_type', label='Types of Jet'),
  Regular(50, 1, 10000, transform=log, name='jet_pt', label='$p_T^{jet}$'),
  Variable([-5, -3, -2.5, -1.3, 0, 1.3, 2.5, 3, 5], name='jet_eta', label='$\\eta^{jet}$'),
  Regular(50, -3.14159, 3.14159, circular=True, name='jet_phi', label='$\\phi^{jet}$'),
  Regular(200, 0, 5, name='pt_response', label='$p_T$ response'),
  storage=Weight()) # Sum: WeightedSum(value=1.16283e+07, variance=1.16283e+07) (WeightedSum(value=1.16328e+07, variance=1.16328e+07) with flow)

In [63]:
out_fit_stats

{'centers': array([0.0125, 0.0375, 0.0625, 0.0875, 0.1125, 0.1375, 0.1625, 0.1875,
        0.2125, 0.2375, 0.2625, 0.2875, 0.3125, 0.3375, 0.3625, 0.3875,
        0.4125, 0.4375, 0.4625, 0.4875, 0.5125, 0.5375, 0.5625, 0.5875,
        0.6125, 0.6375, 0.6625, 0.6875, 0.7125, 0.7375, 0.7625, 0.7875,
        0.8125, 0.8375, 0.8625, 0.8875, 0.9125, 0.9375, 0.9625, 0.9875,
        1.0125, 1.0375, 1.0625, 1.0875, 1.1125, 1.1375, 1.1625, 1.1875,
        1.2125, 1.2375, 1.2625, 1.2875, 1.3125, 1.3375, 1.3625, 1.3875,
        1.4125, 1.4375, 1.4625, 1.4875, 1.5125, 1.5375, 1.5625, 1.5875,
        1.6125, 1.6375, 1.6625, 1.6875, 1.7125, 1.7375, 1.7625, 1.7875,
        1.8125, 1.8375, 1.8625, 1.8875, 1.9125, 1.9375, 1.9625, 1.9875,
        2.0125, 2.0375, 2.0625, 2.0875, 2.1125, 2.1375, 2.1625, 2.1875,
        2.2125, 2.2375, 2.2625, 2.2875, 2.3125, 2.3375, 2.3625, 2.3875,
        2.4125, 2.4375, 2.4625, 2.4875, 2.5125, 2.5375, 2.5625, 2.5875,
        2.6125, 2.6375, 2.6625, 2.6875, 2.7125, 2.737

In [62]:
out_fit_stats = fit_response(out, dataset=dataset, 
                             correction_level="Original", 
                             jet_type="AK4PUPPI",
                             xscale="linear", yscale="linear", 
                             legend_title="Original JEC", hep_args=mc_hep_args,
                             plot=True,
                             filename=None
                            )

/tmp/ipykernel_1297125/919783901.py:17: RuntimeWarning: invalid value encountered in divide
  ave = np.sum(centers * freqs, axis=-1) / total
/tmp/ipykernel_1297125/919783901.py:18: RuntimeWarning: invalid value encountered in divide
  var = np.sum((centers**2) * freqs, axis=-1) / total - (ave ** 2)
/tmp/ipykernel_1297125/919783901.py:21: RuntimeWarning: invalid value encountered in divide
  mean_error = np.where(total > 0, stdev / np.sqrt(total), 0)


In [39]:
def plot_response_stat(pt_bins, eta_bins, response_stat,
                       jet_label="jet", 
                       xscale="linear", yscale="linear", xlim=None, ylim=None,
                       legend_title=None, hep_args=None, filename=None, dpi=100):
    fig, ax = plt.subplots(figsize=(10, 10))
    cmap = cm.seismic_r
    cmap.set_bad(color='gray', alpha=0.2)

    hep.hist2dplot(response_stat, pt_bins, eta_bins, ax=ax, cmap=cmap, norm=colors.CenteredNorm(vcenter=1.0))
    
    # plot pt=40 GeV
    ax.axvline(x=40, color="k", linestyle="dotted")
    
    # plot eta range in eta_bin_plot
    for eta in eta_bins:
        ax.axhline(y=eta, color="k", linestyle="dotted")
        if eta !=0:
            ax.axhline(y=-eta, color="k", linestyle="dotted")
    
    ax.set_xlabel(r"$p_T^{%s}$"%(jet_label))
    ax.set_ylabel(r"$\eta^{%s}$"%(jet_label))
    
    format_plot(fig, ax, xscale=xscale, yscale=yscale, xlim=xlim, ylim=ylim, 
                legend_title=legend_title, legend_args={"title_fontsize":20},
                hep_args=hep_args,
                filename=filename, dpi=dpi)

In [40]:
pt_bins = out["pt_response"].axes["jet_pt"].edges
eta_bins = out["pt_response"].axes["jet_eta"].edges

In [41]:
#%%script false --no-raise-error
plot_response_stat(pt_bins, eta_bins, out_fit_stats["fit_mean"],
                   jet_label="PUPPI",
                   xscale="log",
                   legend_title="Mean", hep_args=hep_args)

NameError: name 'out_fit_stats' is not defined

In [ ]:
out_fit_stats.keys()

In [ ]:
fig, ax = plt.subplots()
pt_bin_centers = out["pt_response"].axes["jet_pt"].centers
plot_stats = ["mean", "median", "fit_mean"]
fmts = "os^"
for i, stat in enumerate(plot_stats):
    stat_err = stat + "_error"
    ax.errorbar(pt_bin_centers, out_fit_stats[stat][:, 3], yerr=out_fit_stats[stat_err][:, 3], 
                fmt=fmts[i], label=stat, alpha=0.5)
ax.axhline(y=1.0, linestyle="dotted", color="k")
ax.legend()
ax.set_xscale("log")
ax.set_ylim(-5, 6)
ax.legend(title=r"$0 < \eta$ < 1.3", loc=3)
ax.set_xlabel(r"AK4PUPPI $p_T$")
correction_level = format_correction_level("Original")
off_jet_label = "AK4PUPPI"
on_jet_label = "AK4HLT"
jet_type = "AK4PUPPI"
off_correction_level, on_correction_level = extract_correction_level(correction_level)
off_jet_pt_axis_label = "\ ".join([off_jet_label, "p_T", "({})".format(off_correction_level)])
on_jet_pt_axis_label = "\ ".join([on_jet_label, "p_T", "({})".format(on_correction_level)])
if jet_type.startswith(off_jet_label):
    jet_type_correction_level = off_correction_level
else:
    jet_type_correction_level = on_correction_level
jet_type_pt_axis_label = "\ ".join([*jet_type.split("_"), "p_T", "({})".format(jet_type_correction_level)])
ax.set_xlabel(r"${}$".format(jet_type_pt_axis_label))
ax.set_ylabel(r"$R = \frac{%s}{%s}$"%(str(on_jet_pt_axis_label), str(off_jet_pt_axis_label)))
ax.grid()
hep.cms.label(**mc_hep_args)

In [ ]:
out["arguments"]